Исходно в Colab очень низкая версия библиотеки xgboost. Прийдется заменить.

ВНИМАНИЕ!!! Не забудьте ответить на запрос об отмене удаления


In [ ]:
%pip uninstall xgboost
%pip install xgboost==1.6.2
import xgboost as xgb
xgb.__version__

Found existing installation: xgboost 0.90
Uninstalling xgboost-0.90:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/xgboost-0.90.dist-info/*
    /usr/local/lib/python3.7/dist-packages/xgboost/*
    /usr/local/xgboost/libxgboost.so
Proceed (y/n)? y
  Successfully uninstalled xgboost-0.90
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 255.9 MB 49 kB/s 


'1.6.2'

In [ ]:
import numpy as np
import random
from tensorflow.keras.models import Model, load_model  # Импортируем модели keras: Model
from tensorflow.keras.layers import Input, Conv2DTranspose, concatenate, Activation, SpatialDropout2D, MaxPooling2D, \
    AveragePooling2D, Conv2D, BatchNormalization  # Импортируем стандартные слои keras
from tensorflow.keras import backend as K  # Импортируем модуль backend keras'а
from tensorflow.keras.optimizers import Adam  # Импортируем оптимизатор Adam
from tensorflow.keras import \
    utils  # Импортируем модуль utils библиотеки tensorflow.keras для получения OHE-представления
from keras import regularizers
from keras.callbacks import Callback
import tensorflow as tf
import os
import os.path
from tqdm.auto import tqdm
import glob

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.utils import to_categorical
from tensorflow.test import is_gpu_available

from xgboost import XGBClassifier
from xgboost.callback import EarlyStopping

from sklearn.metrics import accuracy_score


import warnings
warnings.filterwarnings("ignore")

# Модуль предполагает запуск на разных устройствах. Этот блок кода как раз 
# определяет, откуда он вызван.
Colab = True
try:
    from google.colab import drive
except:
    Colab = False

if Colab:
    from google.colab import drive

    # Подключаем Google drive
    drive.mount('/content/drive')
    CrPath = "/content/drive/MyDrive/Uinnopolis/"

    import sys
    sys.path.append('/content/drive/MyDrive/Uinnopolis')
else:
    Acer = not os.path.exists("E:/Uinnopolis/")
    CrPath = "C:/Uinnopolis/" if Acer else "E:/Uinnopolis/"

import os, glob
from Libs import *



Mounted at /content/drive


Алгоритм определения лучших признаков. Работает так.

Сперва обучаем классификатор на тренеровочном наборе. После чего классификатор сам рассчитает рейтинг признаков.
Сортируем его, и дальше последовательно добавляем их по одному ко входному набору, начиная с самых значимых. Если при добавлении новой позиции результат распознавания улучшается, данный признак добавляется в набор данных

In [ ]:
def XGSearchFeatures(SearchFtrX, SearchFtrY, X, Y, ValidX, ValidY, XTest, From = 0, InputMask = None, GPU = 0, UseMetric="mlogloss"):
    # метод обработки модели
    def StartModel(Ft, PrintI):
        model = XGBClassifier(learning_rate=0.03,
                              n_estimators=50000,
                              max_depth=6,
                              min_child_weight=6,
                              max_bin=100,
                              gamma=0,
                              subsample=0.6,
                              colsample_bytree=0.6,
                              reg_alpha=0.005,
                              objective='binary:logistic',
                              nthread=6,
                              scale_pos_weight=1,
                              seed=int(65),
                              tree_method=CrMethod,
                              random_state=65,
                              verbose=0)
        
        # Call back обеспечивает своевременную остановку при начале переобучения
        es = EarlyStopping(
            rounds=100,
            save_best=True,
            maximize=False,
            data_name="validation_0",
            metric_name=UseMetric
        )

        VX = ValidX[:, Ft]
        eval_set = [(VX, ValidY)]

        model.fit(X[:, Ft], Y, eval_metric=UseMetric, eval_set=eval_set,
                  verbose=0, callbacks=[es])

        # make predictions for test data
        y_pred = model.predict(VX)

        accuracy = accuracy_score(ValidY, y_pred)
        print(PrintI, 'Accuracy: %.8f%%' % (accuracy * 100.0))

        return accuracy, model

    # задача важная. Поэтому проверяем, что все как надо
    print(0, ValidX.shape)
    print(1, ValidY.shape)
    print(2, X.shape)
    print(3, Y.shape)
    print(GPU)

    RA = [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2, 5e-2, 1e-1, 5e-1, 1, 5, 10, 50, 100]
    Methods = ['exact', 'approx', 'hist']
    CrMethod = 'exact'

    if GPU:  # and CrMethod == 'hist':
        CrMethod = 'gpu_hist'

    # Для статистики просчитваем исходный набор данных, без дополнительных признаков
    StartModel(np.arange(X.shape[-1])[-139:], 'Исходный набор признаков (временные метки и производная')
    _a, model = StartModel(np.arange(X.shape[-1]), 'Полный набор признаков')
    
    # Ранжируем признаки входного сигнала по значимости
    
    f = model.feature_importances_ # Поличили значимость признаков
    np.save(f'{CrPath}feature_importances.npy', f)
    print('Ready feature_importances.npy', f.shape)
    fInd = np.argsort(f)[::-1]
    CrIndex = fInd.copy()
    LastAccuracy = 0

    # И теперь запускаем жадный алгоритм подбора лучших. Оставляем только те
    # признаки, добавление которых улучшает результат 

    #Result = 'Temp_Result3_399.npy'
    #From = 400
    #LastAccuracy = 0.99
    From = 0
    LastAccuracy = 0
    if not os.path.isfile(f'{CrPath}Ftr_Result.npy'):
        Result = [0] * len(f)

        i = From # даем возможность возобновиться после прерывания

        for n in range(len(f) - From):
            # считаем модель для текущего количества признаков
            accuracy, m = StartModel(fInd[:i+1], f'{i}/{n+From}')

            if LastAccuracy > accuracy:
                # Если признак ухудшает результат - его не берем
                fInd = np.delete(fInd, [i])
                Result[n] = 0
                print('Пропускаем', 'accuracy остается ', LastAccuracy)
                
            else:
                LastAccuracy = accuracy
                Result[n] = accuracy
                i+= 1

            if n % 100 == 99: # Промежуточное сохранение
                Ft = np.array(Result)
                np.save(f'{CrPath}Temp_Result3_{n}.npy', Ft)
            #print(i, 'Accuracy: %.8f%%' % (accuracy * 100.0))


        # Финальное сохранение файла восстановления
        Ft = np.array(Result)
        np.save(f'{CrPath}Ftr_Result.npy', Ft)

    else:
        Ft = np.load(f'{CrPath}Ftr_Result.npy')

    # Теперь у нас есть значения accuracy для каждого набора фич от начала
    # списка лучших до текущей ( за вычетом удаленных). Удаленные в этом списке 
    # помечены 0     

    
    HallIndexes = np.arange(len(Ft))

    # определяем лучшие позиции
    TheBest = np.max(Ft)
    BestIndex = np.argmax(Ft)

    fInd = fInd[Ft > 0] # массив всех непропущенных признаков
    # Сохранили его - собственно, это главный итог работы этой задачи 
    np.save(f'{CrPath}FeatureIndex3.npy', fInd[:BestIndex])

    # Но теперь немного подсоберем статистики - чтобы увидеть, что все не зря
    
    # Во-первых, просто убеждаемся, что все верно. Результат должен совпасть с лучшим
    StartModel(fInd[:BestIndex], 'Финальный тест')

    print('Лучший результат', TheBest)
    print('Полный результат', CrReiting)


In [ ]:
ResX1, ResY = ReadCsv(CrPath, DelZeros=True, SortCtg=False, Train=False, RetPrc=True, PostProc=True)

_x, Y = ReadCsv(CrPath, DelZeros=True, SortCtg=False, Train=True, RetPrc=True, PostProc=True)
y_valid = Y[:100]
y_train = Y[100:]

K = 20
y_train = np.tile(y_train, [K, 1])

XValid = np.load(f'{CrPath}Input/Valid_100_FullFields.npy')
X = np.load(f'{CrPath}Input/Train_100_FullFields.npy')

ResX0, YTest = ReadCsv(CrPath, DelZeros=True, SortCtg=False, Train=False, RetPrc=True, PostProc=True)
Ftr_Test = np.load(f'{CrPath}Input/Ftr_test.npy')
XTest = np.concatenate((Ftr_Test, ResX0), -1)

y_valid = np.tile(y_valid, [K, 1])

fInd = np.load(f'{CrPath}Input/FeatureIndex0.npy')
XTest = XTest[:, fInd]

XGSearchFeatures(X, y_train[:, 1], X, y_train[:, 1], XValid, y_valid[:, 1:2], XTest = None, GPU=is_gpu_available(), UseMetric="mlogloss")

0 (2000, 928)
1 (2000, 1)
2 (94600, 928)
3 (94600,)
True
[11:23:50] WARNING: ../src/learner.cc:627: 
Parameters: { "scale_pos_weight", "verbose" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


Исходный набор признаков (временные метки и производная Accuracy: 96.20000000%
[11:24:21] WARNING: ../src/learner.cc:627: 
Parameters: { "scale_pos_weight", "verbose" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


Полный набор признаков Accuracy: 96.15000000%
Ready feature_importances.npy (928,)
[11:25:58] WARNING: ../sr

KeyboardInterrupt: ignored